# pynq驱动AR0134摄像头，LCD显示，OpenCV处理

初始化库

In [1]:
from pynq import Overlay
from pynq.lib.iic import *
from ov5640_cfg import *
from ar0134_cfg import *
from pynq.lib.video import *
from time import sleep
import dma
import os
import shutil
import cv2
from PIL import Image
import IPython
import numpy as np
import time
from pynq.lib import AxiGPIO

cv_ov5640 = Overlay("cv_ov5640.bit")

复位LCD

In [2]:
led_instance = cv_ov5640.ip_dict['axi_gpio_1']
led = AxiGPIO(led_instance).channel1

In [3]:
led[0].write(0x0)

In [4]:
led[0].write(0x1)

初始化摄像头

In [5]:
iic = AxiIIC(cv_ov5640.ip_dict['axi_iic_0']);
address = 0x10;
length = 4;
time.sleep(1)
for config in ar0134_cfg:
    tmp1 = config[0] >> 8;
    tmp2 = config[0] & 0xff;
    #iic.send(address, bytes([tmp1, tmp2]), length)
    #time.sleep(0.0005)
    tmp3 = config[1] >> 8;
    tmp4 = config[1] & 0xff;
    iic.send(address, bytes([tmp1, tmp2, tmp3, tmp4]), length)
    if config[0] == 0x301A or config[0] == 0x30D4 or config[0] == 0x30B0:
        time.sleep(0.05)
    time.sleep(0.005)
    
    #iic.wait()
    
    

Sensor Demosaic ip配置，raw转rgb

In [6]:
sensor_demosaic = cv_ov5640.v_demosaic_0
sensor_demosaic.write(0x0010,1280)
sensor_demosaic.write(0x0018,720)
sensor_demosaic.write(0x0028,0x03)
sensor_demosaic.write(0x0000,0x81)



rgb顺序转换，图像大小转换，适应屏幕

In [7]:
frame_resize = cv_ov5640.frame_resize_0
frame_resize.write(0x10, 720)
frame_resize.write(0x18, 1280)
frame_resize.write(0x20, 0)
frame_resize.write(0x00, 0x81)

rgb2gbr = cv_ov5640.RGB2GBR_0
rgb2gbr.write(0x10, 720)
rgb2gbr.write(0x18, 1280)
rgb2gbr.write(0x20, 0)
rgb2gbr.write(0x00, 0x81)

开启vdma，一个用于LCD显示，一个用于向ps输送图像，OpenCV处理

In [8]:
vdma0 = cv_ov5640.axi_vdma_0
vdma1 = cv_ov5640.axi_vdma_1
vdma0.writechannel.mode = VideoMode(1280, 720, 24)
vdma1.readchannel.mode = vdma0.writechannel.mode
vdma0.readchannel.mode = vdma0.writechannel.mode
vdma1.readchannel.mode = VideoMode(800, 480, 24)
vdma1.writechannel.mode = vdma1.readchannel.mode
vdma0.readchannel.tie(vdma0.writechannel)
vdma1.readchannel.tie(vdma1.writechannel)
vdma0.writechannel.start()
vdma0.readchannel.start()
vdma1.writechannel.start()
vdma1.readchannel.start()

得到Mat格式图像，OpenCV处理

In [ ]:
while True:
    while True:
        frame = vdma0.readchannel.readframe()
        if(frame.max() != 0):
            break;

    image_pil = Image.fromarray(frame)
    frame =0
    img_cv = np.asarray(image_pil) #得到mat类型的数据
    img_cv = cv2.resize(img_cv, (192,168))
    #利用img_cv进行opencv处理
    #cv2.rectangle(img_cv,(int(0),int(0)),(int(100),int(100)),(0,255,0),3)


    #将opencv数据转回PIL数据，显示
    #img1 = Image.fromarray(cv2.cvtColor(img_cv,cv2.COLOR_RGB2BGR))
    img1 = Image.fromarray(img_cv)
    #b, g, r = img1.split()
    #img1 = Image.merge('RGB', [g,b,r])
    IPython.display.clear_output(1)
    display(img1)
    time.sleep(0.2)
    #vdma0.writechannel.writeframe(img1) 
    #vdma1.writechannel.writeframe(image)  

后面的没有用，是调试时留下的，可以参考，人脸检测及pwm生成

In [4]:
gaussianBlur = cv_ov5640.ov5640_GaussianBlur_0
gaussianBlur.write(0x10, 720)
gaussianBlur.write(0x18, 1280)
gaussianBlur.write(0x20, 0)
gaussianBlur.write(0x00, 0x81)
vdma0 = cv_ov5640.axi_vdma_0
vdma1 = cv_ov5640.axi_vdma_1
vdma0.writechannel.mode = VideoMode(192, 168, 8)
#vdma1.readchannel.mode = vdma0.writechannel.mode
vdma0.readchannel.mode = vdma0.writechannel.mode
vdma1.readchannel.mode = VideoMode(800, 480, 24)
vdma1.writechannel.mode = vdma1.readchannel.mode
vdma0.readchannel.tie(vdma0.writechannel)
vdma1.readchannel.tie(vdma1.writechannel)
vdma0.writechannel.start()
vdma0.readchannel.start()
vdma1.writechannel.start()
vdma1.readchannel.start()

In [3]:
pwm = cv_ov5640.pwm_generate_0
pwm.write(0x00, 1000)
pwm.write(0x04, 500)
pwm.write(0x08, 10000)
pwm.write(0x0c, 5000)
pwm.write(0x10, 10000)
pwm.write(0x14, 5000)
pwm.write(0x18, 10000)
pwm.write(0x1c, 5000)



In [6]:
classifier=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
while(True):
    while(True):
        frame = vdma0.readchannel.readframe()
        if(frame.max() != 0):
            break;

    time_start=time.time()
    image_pil = Image.fromarray(frame)
    #frame =0
    img_cv = np.asarray(image_pil) #得到mat类型的数据
    #tempImg = cv2.resize(img_cv, (320,240))  # 缩小图像为320x240尺寸

    #image=cv2.cvtColor(img_cv,cv2.COLOR_BGR2GRAY)
    #cv2.equalizeHist(image)
    divisor=4
    h=168
    w=192
    minSize=(w//divisor,h//divisor)
    faceRects=classifier.detectMultiScale(img_cv,1.2,2,cv2.CASCADE_SCALE_IMAGE,minSize)
    #end = time.time()

    if len(faceRects)>0:
        for faceRect in faceRects:
            x,y,w,h=faceRect
            #cv2.circle(img_cv,(x+w//2,y+h//2),min(w//2,h//2),(255,0,0))
            #cv2.circle(img_cv,(x+w//4,y+h//4),min(w//8,h//8),(255,0,0))
            #cv2.circle(img_cv,(x+3*w//4,y+h//4),min(w//8,h//8),(255,0,0))
            #cv2.rectangle(img_cv,(x+3*w//8,y+3*h//4),(x+5*w//8,y+7*h//8),(255,0,0))
    #     end = time.time()
    time_end=time.time()
    #img = arrayShow( tempImg)
    #display(img)
    #print(end - start)
    #time.sleep(0.1)
    img3 = Image.fromarray(cv2.cvtColor(img_cv,cv2.COLOR_GRAY2RGB))
        #img3 = Image.fromarray(img_cv_1)
    IPython.display.clear_output(1)
    display(img3)
    print('totally cost',time_end-time_start)

KeyboardInterrupt: 

In [10]:
vdma0.writechannel.stop()
vdma0.readchannel.stop()
vdma1.writechannel.stop()
vdma1.readchannel.stop()